In [1]:
from latentneural.lorenz import LorenzGenerator, ic
from latentneural.data import DataManager
from latentneural.utils import logger
import numpy as np
import tensorflow as tf
import os
import yaml
from copy import deepcopy

In [2]:
step = 0.01 # Time step in the simulatio (in seconds)
stop = 1 # Final time for the simulation (in seconds)
neurons = 30 # Number of neurons
base_rate = 5 # Base frequency (in Hz)
latent_dim = 3 # Lorenz latent variables encoded in neural activity (relevant+irrelevant)
relevant_dim = 2 # Lorenz latent variables encoded in behavioural activity (relevant)
behaviour_dim = 4 # Behavioural channels
conditions = 1 # Number of conditions to be tested (each condition has different weight matrices to convert latent variables into firing rates and behaviour)
trials = 1000 # Number of trials to be tested for each condition
initial_conditions = ic.uniform(low=-10, high=10) # Sample initial conditions of Lorenz system from uniform distribution
behaviour_sigma=1 # Standard deviation noise added to behaviour
seed=0

In [3]:
with open(os.path.join('..', 'latentneural', 'data', 'storage', \
                       'lorenz', 'grid', 'settings_template.tndm.long.yaml'), 'r') as f:
   training_settings_template = yaml.safe_load(f)
print(training_settings_template)

{'data': {'behavioural_keys': {'test': 'test_behaviours', 'train': 'train_behaviours', 'validation': 'valid_behaviours'}, 'dataset_filename': 'dataset.h5', 'directory': './latentneural/data/storage/lorenz/grid/{{name}}', 'latent_keys': {'test': 'test_latent', 'train': 'train_latent', 'validation': 'valid_latent'}, 'metadata_filename': 'metadata.json', 'neural_keys': {'test': 'test_data', 'train': 'train_data', 'validation': 'valid_data'}}, 'model': {'settings': {'default_layer_settings': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 0.1}, 'type': 'l2'}}, 'encoded_dim': 64, 'relevant_factors': 2, 'irrelevant_factors': 1, 'full_logs': False, 'layers': {'behavioural_dense': {'behaviour_sigma': 2.0, 'behaviour_type': 'synchronous', 'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regular

In [4]:
np.random.seed(seed)
tf.random.set_seed(seed)

for behaviour_sigma in [0.5, 1, 2]:
    for base_rate in [5, 10, 15]:
        for train_num in [50, 100, 200]:
            valid_num = train_num
            test_num = trials

            dirname = 'train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma)
            data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', 'grid', \
                                    dirname)
            training_settings = deepcopy(training_settings_template)
            training_settings['data']['directory'] = \
                training_settings['data']['directory'].replace('{{name}}', dirname)
            training_settings['output']['directory'] = \
                training_settings['output']['directory'].replace('{{name}}', dirname)
            
            # keeping ratio of behaviour sigmas constant
            # training_settings['model']['settings']['layers']['behavioural_dense']['behaviour_sigma'] *= \
            #     (behaviour_sigma / 0.5)
            
            with open(os.path.join(data_dir, 'training_settings.tndm.long.21.yaml'), 'w') as fp:
                yaml.dump(training_settings, fp)
            logger.info('Training settings stored.')
            logger.info('Stored train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma))

[16:54:15.837] INFO [latentneural.utils.logging.<module>:26] Training settings stored.
[16:54:15.839] INFO [latentneural.utils.logging.<module>:27] Stored train_trials=50|baselinerate=5|behaviour_sigma=0.5
[16:54:15.850] INFO [latentneural.utils.logging.<module>:26] Training settings stored.
[16:54:15.851] INFO [latentneural.utils.logging.<module>:27] Stored train_trials=100|baselinerate=5|behaviour_sigma=0.5
[16:54:15.864] INFO [latentneural.utils.logging.<module>:26] Training settings stored.
[16:54:15.866] INFO [latentneural.utils.logging.<module>:27] Stored train_trials=200|baselinerate=5|behaviour_sigma=0.5
[16:54:15.877] INFO [latentneural.utils.logging.<module>:26] Training settings stored.
[16:54:15.878] INFO [latentneural.utils.logging.<module>:27] Stored train_trials=50|baselinerate=10|behaviour_sigma=0.5
[16:54:15.890] INFO [latentneural.utils.logging.<module>:26] Training settings stored.
[16:54:15.891] INFO [latentneural.utils.logging.<module>:27] Stored train_trials=100|b